In [1]:
from drone_detector.utils import *
from drone_detector.imports import *
import os
from drone_detector.metrics import *
import warnings
warnings.filterwarnings("ignore")
sys.path.append('..')
from src.postproc_functions import *

# Virtual plot level object detection results

# Hiidenportti test set

As Hiidenportti test set is so small, we can run predictions here if needed.

## No overlap, no post-processing

In [4]:
from drone_detector.engines.detectron2.predict import predict_instance_masks
raw_path = Path('../../data/raw/hiidenportti/virtual_plots/buffered_test/images')
test_rasters = [raw_path/f for f in os.listdir(raw_path) if f.endswith('tif')]

Template folder has the following structure:

```
template_folder
|-predicted_vectors
|-raster_tiles
|-vector_tiles
|-raw_preds
```

Where `raster_tiles` and `vector_tiles` are symbolic links pointing to corresponding data directories, and `predicted_vectors` and `raw_preds` are empty folders for predictions.

In [5]:
pred_outpath = Path('../results/hp_unprocessed_256/')
if not os.path.exists(pred_outpath):
    shutil.copytree('../results/template_folder/', pred_outpath)

In [7]:
for t in test_rasters:
    outfile_name = pred_outpath/f'raw_preds/{str(t).split("/")[-1][:-4]}.geojson'
    predict_instance_masks(path_to_model_config='../models/hiidenportti/mask_rcnn_R_50_FPN_3x_256/config.yaml', 
                           path_to_image=str(t),
                           outfile=str(outfile_name),
                           processing_dir='temp',
                           tile_size=256,
                           tile_overlap=0,
                           smooth_preds=False,
                           use_tta=True,
                           coco_set='../../data/processed/hiidenportti/hiidenportti_valid.json',
                           postproc_results=False)

Reading and tiling ../../data/raw/hiidenportti/virtual_plots/buffered_test/images/104_49_Hiidenportti_Chunk5_orto.tif to 256x256 tiles with overlap of 0px


0it [00:00, ?it/s]

Loading model
Starting predictions


  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

Loading model
Starting predictions


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

276 polygons before non-max suppression
276 polygons after non-max suppression
Removing intermediate files


In [8]:
raw_res_path = pred_outpath
truth_shps = sorted([raw_res_path/'vector_tiles'/f for f in os.listdir(raw_res_path/'vector_tiles')])
raw_shps = sorted([raw_res_path/'raw_preds'/f for f in os.listdir(raw_res_path/'raw_preds')])
rasters = sorted([raw_res_path/'raster_tiles'/f for f in os.listdir(raw_res_path/'raster_tiles')])

"Raw" predictions are modified as such:
1. Invalid polygons are fixed to be valid polygons. MultiPolygon masks are replaced with the largest single polygon of the multipoly.
2. Extent is clipped to be same as the corresponding ground truth data
3. Label numbering is adjusted
4. Polygons with area less than 16² pixels are discarded

In [9]:
for p, t in zip(raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.04**2]
    temp_pred.to_file(raw_res_path/'predicted_vectors'/p.name)

In [10]:
pred_shps = sorted([raw_res_path/'predicted_vectors'/f for f in os.listdir(raw_res_path/'predicted_vectors')])

In [11]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [12]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [13]:
preds.shape, truths.shape

((3647, 4), (1547, 6))

In [14]:
preds.label.value_counts()

2    3163
1     484
Name: label, dtype: int64

In [15]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [16]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.450745
uprightwood    0.383135
dtype: float64

In [17]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

raw_coco_eval = GisCOCOeval(raw_res_path, raw_res_path, 
                            None, None, deadwood_categories)

In [18]:
raw_coco_eval.prepare_data(gt_label_col='layer')

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [19]:
raw_coco_eval.prepare_eval()

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!


In [20]:
raw_coco_eval.coco_eval.params.maxDets = [1000, 10000]

In [21]:
raw_coco_eval.evaluate()


Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.79s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.224
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.469
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.134
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.280
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.800
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.379
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.316
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.414
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.800

Evaluating for 

In [22]:
fp_cols = [f'FP_{np.round(i, 2)}' for i in np.arange(0.5, 1.04, 0.05)]
tp_cols = [f'TP_{np.round(i, 2)}' for i in np.arange(0.5, 1.03, 0.05)]

In [23]:
from tqdm import tqdm
tqdm.pandas()

In [24]:
tp_truths = truths.copy()
tp_truths.rename(columns={'groundwood':'label'}, inplace=True)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

100%|██████████| 3647/3647 [00:22<00:00, 158.82it/s]


In [25]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,2243,920,3163
uprightwood,289,195,484
All,2532,1115,3647


In [26]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,343,920,1263
uprightwood,89,195,284
All,432,1115,1547


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [106]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(915/2253):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(915/1263):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.41
Recall for fallen deadwood with IoU threshold of 0.5 is 0.72


In [107]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(176/263):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(176/284):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.67
Recall for standing deadwood with IoU threshold of 0.5 is 0.62


In [108]:
print(f'Overall precision with IoU threshold of 0.5 is {(1091/2516):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(1091/1547):.2f}')

Overall precision with IoU threshold of 0.5 is 0.43
Overall recall with IoU threshold of 0.5 is 0.71


## Half patch overlap and edge filtering

For this postprocessing method, mosaics are tiled so that the sliding window moves half tile lenght. For example, when moving row-wise, the first bottom-left coordinates are (0,0), and next ones (256,0), (512,0)... and same  is done column-wise. We discard all predicted polygons whose centroid point is not within the half-overlap area. For instance, for first tile (bottom left (0,0)), the x-coordinate must be between 128 and 384, for second tile (256,0) between 384 and 640, and likewise for y-coordinates.

The images used for predictions are buffered so that the whole area is covered, considering the discarding process.

In [27]:
pred_outpath = Path('../results/hp_overlap_filter_256/')
if not os.path.exists(pred_outpath):
    shutil.copytree('../results/template_folder/', pred_outpath)

In [28]:
raw_path = Path('../../data/raw/hiidenportti/virtual_plots/buffered_test/images')
test_rasters = [raw_path/f for f in os.listdir(raw_path) if f.endswith('tif')]

for t in test_rasters:
    outfile_name = pred_outpath/f'raw_preds/{str(t).split("/")[-1][:-4]}.geojson'
    predict_instance_masks(path_to_model_config='../models/hiidenportti/mask_rcnn_X_101_32x8d_FPN_3x/config.yaml', 
                           path_to_image=str(t),
                           outfile=str(outfile_name),
                           processing_dir='temp',
                           tile_size=256,
                           tile_overlap=128,
                           smooth_preds=False,
                           use_tta=True,
                           coco_set='../../data/processed/hiidenportti/hiidenportti_valid.json',
                           postproc_results=True)

Reading and tiling ../../data/raw/hiidenportti/virtual_plots/buffered_test/images/104_49_Hiidenportti_Chunk5_orto.tif to 256x256 tiles with overlap of 128px


0it [00:00, ?it/s]

Loading model
Starting predictions


  0%|          | 0/1222 [00:00<?, ?it/s]

  0%|          | 0/1222 [00:00<?, ?it/s]

...

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

1065 polygons before edge area removal


  0%|          | 0/132 [00:00<?, ?it/s]

253 polygons before non-max suppression
253 polygons after non-max suppression
Removing intermediate files


Modify as previously.

In [29]:
hp_res_path = pred_outpath
truth_shps = sorted([hp_res_path/'vector_tiles'/f for f in os.listdir(hp_res_path/'vector_tiles')])
hp_raw_shps = sorted([hp_res_path/'raw_preds'/f for f in os.listdir(hp_res_path/'raw_preds')])
rasters = sorted([hp_res_path/'raster_tiles'/f for f in os.listdir(hp_res_path/'raster_tiles')])

In [30]:
for p, t in zip(hp_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.04**2]
    temp_pred.to_file(hp_res_path/'predicted_vectors'/p.name)

In [31]:
pred_shps = sorted([hp_res_path/'predicted_vectors'/f for f in os.listdir(hp_res_path/'predicted_vectors')])

Collate all predictions into single dataframes

In [32]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [33]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [36]:
preds.layer.value_counts()

groundwood     3499
uprightwood     321
Name: layer, dtype: int64

In [37]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [38]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.441568
uprightwood    0.438524
dtype: float64

In [39]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

hp_coco_eval = GisCOCOeval(hp_res_path, hp_res_path, 
                            None, None, deadwood_categories)

In [40]:
hp_coco_eval.prepare_data(gt_label_col='layer')

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [41]:
hp_coco_eval.prepare_eval()

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!


In [42]:
hp_coco_eval.coco_eval.params.maxDets = [1000, 10000]

In [43]:
hp_coco_eval.evaluate()


Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.51s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.262
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.503
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.254
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.172
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.344
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.370
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.311
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.407
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.000

Evaluating for 

In [44]:
tp_truths = truths.copy()
tp_truths.rename(columns={'groundwood':'label'}, inplace=True)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

100%|██████████| 3820/3820 [00:22<00:00, 169.50it/s]


In [134]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,1126,1009,2135
uprightwood,68,197,265
All,1194,1206,2400


In [135]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,254,1009,1263
uprightwood,87,197,284
All,341,1206,1547


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [136]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(1009/2135):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(1009/1263):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.47
Recall for fallen deadwood with IoU threshold of 0.5 is 0.80


In [137]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(197/265):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(197/284):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.74
Recall for standing deadwood with IoU threshold of 0.5 is 0.69


In [138]:
print(f'Overall precision with IoU threshold of 0.5 is {(1206/2400):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(1206/1547):.2f}')

Overall precision with IoU threshold of 0.5 is 0.50
Overall recall with IoU threshold of 0.5 is 0.78


## Overlap, edge filtering and mask merging

Mask merging is built on previous predictions. In this step, for each polygon we check whether the ratio between intersection with any other polygon of the same class and the area of the polygon is more than 0.2. If yes, the polygon is merged to the other polygon with which it had intersection-over-area ratio.

In [45]:
merge_outpath = Path('../results/hp_merge_256/')
if not os.path.exists(merge_outpath):
    shutil.copytree('../results/template_folder/', merge_outpath)

Two iterations of merging is usually enough.

In [46]:
for r in pred_shps:
    gdf_temp = gpd.read_file(r)
    standing = gdf_temp[gdf_temp.label==1].copy()
    fallen = gdf_temp[gdf_temp.label==2].copy()
    standing = merge_polys(standing, 0.2)
    fallen = merge_polys(fallen, 0.2)
    standing = merge_polys(standing, 0.2)
    fallen = merge_polys(fallen, 0.2)
    gdf_merged = pd.concat((standing, fallen))
    gdf_merged.to_file(merge_outpath/'predicted_vectors'/r.name, driver='GeoJSON')
    gdf_merged = None
    gdf_temp = None

17it [00:00, 645.22it/s]
193it [00:00, 341.44it/s]
17it [00:00, 704.52it/s]
145it [00:00, 501.52it/s]
41it [00:00, 654.93it/s]
570it [00:02, 278.09it/s]
41it [00:00, 615.90it/s]
443it [00:01, 408.39it/s]
97it [00:00, 596.05it/s]
996it [00:04, 229.65it/s]
97it [00:00, 620.32it/s]
761it [00:02, 325.44it/s]
148it [00:00, 579.53it/s]
1549it [00:08, 191.57it/s]
148it [00:00, 584.57it/s]
1234it [00:04, 250.60it/s]
18it [00:00, 743.65it/s]
191it [00:00, 381.87it/s]
18it [00:00, 752.84it/s]
160it [00:00, 536.06it/s]


In [47]:
merged_coco_eval = GisCOCOeval(merge_outpath, merge_outpath, None, None, deadwood_categories)
merged_coco_eval.prepare_data(gt_label_col='layer')
merged_coco_eval.prepare_eval()
merged_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.52s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.262
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.503
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.254
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.172
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.344
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.338
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.285
 Average Recall     (A

This usually worsens the results a bit, but the produced results are better suited for deriving forest charasteristics.

In [67]:
preds.reset_index(drop=True, inplace=True)
standing = merge_polys(preds[preds.label == 1].copy(), 0.2)
fallen = merge_polys(preds[preds.label == 2].copy(), 0.2)
standing = merge_polys(standing, 0.2)
fallen = merge_polys(fallen, 0.2)
preds_merged = pd.concat((standing, fallen))

321it [00:00, 487.01it/s]
3499it [00:31, 111.58it/s]
321it [00:00, 492.01it/s]
2743it [00:18, 151.29it/s]


In [68]:
preds_merged['layer'] = preds_merged.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)
preds_merged.layer.value_counts()

groundwood     2668
uprightwood     320
Name: layer, dtype: int64

In [50]:
tp_truths = truths.copy()
tp_truths.rename(columns={'groundwood':'label'}, inplace=True)
truth_sindex = tp_truths.sindex
fp_preds_merged = preds_merged.copy()
pred_sindex = fp_preds_merged.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds_merged, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds_merged[fp_cols] = fp_preds_merged.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds_merged, pred_sindex),
                                            axis=1, result_type='expand')

100%|██████████| 2988/2988 [00:17<00:00, 168.41it/s]


In [56]:
pd.crosstab(fp_preds_merged.layer, fp_preds_merged['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,1767,901,2668
uprightwood,141,179,320
All,1908,1080,2988


In [57]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,362,901,1263
uprightwood,105,179,284
All,467,1080,1547


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [148]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(912/1665):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(912/1263):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.55
Recall for fallen deadwood with IoU threshold of 0.5 is 0.72


In [149]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(194/258):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(194/284):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.75
Recall for standing deadwood with IoU threshold of 0.5 is 0.68


In [150]:
print(f'Overall precision with IoU threshold of 0.5 is {(1106/1923):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(1106/1547):.2f}')

Overall precision with IoU threshold of 0.5 is 0.58
Overall recall with IoU threshold of 0.5 is 0.71


As the postprocessing merges data instead of dropping less certain predictions, total area and IoU remain the same as in previous step.

In [53]:
preds_merged.to_file('../results/hiidenportti/merged_all_256.geojson')

# Full Evo dataset

Running predictions for Evo dataset takes so much time that it has been done separately. 

## No overlap, no post-processing

In [151]:
spk_raw_res_path = Path('../results/spk_raw/')
truth_shps = sorted([spk_raw_res_path/'vector_tiles'/f for f in os.listdir(spk_raw_res_path/'vector_tiles')])
spk_raw_shps = sorted([spk_raw_res_path/'raw_preds'/f for f in os.listdir(spk_raw_res_path/'raw_preds')])
rasters = sorted([spk_raw_res_path/'raster_tiles'/f for f in os.listdir(spk_raw_res_path/'raster_tiles')])

In [152]:
for p, t in zip(spk_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.0485**2]
    temp_pred.to_file(spk_raw_res_path/'predicted_vectors'/p.name)

In [153]:
pred_shps = sorted([spk_raw_res_path/'predicted_vectors'/f for f in os.listdir(spk_raw_res_path/'predicted_vectors')])

In [154]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [155]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [156]:
preds.shape, truths.shape

((7017, 4), (5334, 4))

In [157]:
preds.layer.value_counts()

groundwood     5663
uprightwood    1354
Name: layer, dtype: int64

In [158]:
truths.rename(columns={'label':'layer'}, inplace=True)

In [159]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [160]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.453250
uprightwood    0.585433
dtype: float64

In [161]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

spk_raw_coco_eval = GisCOCOeval(spk_raw_res_path, spk_raw_res_path, 
                           None, None, deadwood_categories)
spk_raw_coco_eval.prepare_data(gt_label_col='label')
spk_raw_coco_eval.prepare_eval()
spk_raw_coco_eval.coco_eval.params.maxDets = [1000, 10000]
spk_raw_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.08s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.44s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.259
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.515
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.241
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.096
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.539
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.191
 Average Recal

In [163]:
tp_truths

,layer,vplot_id,conservation,geometry
0,groundwood,10_1128,1,"POLYGON ((402535.360 6787757.955, 402535.185 6787758.116, 402534.943 6787758.170, 402534.230 6787758.385, 402533.827 6787758.587, 402533.477 6787758.815, 402533.195 6787758.829, 402532.536 6787758.667, 402532.536 6787758.775, 402532.536 6787759.124, 402532.644 6787759.407, 402532.926 6787759.837, 402533.423 6787759.447, 402533.706 6787759.071, 402535.387 6787758.345, 402535.360 6787757.955))"
1,uprightwood,10_1128,1,"POLYGON ((402539.738 6787761.071, 402539.848 6787760.767, 402540.118 6787760.582, 402540.540 6787760.565, 402540.776 6787760.295, 402540.895 6787759.898, 402540.827 6787759.374, 402540.633 6787758.986, 402539.898 6787758.944, 402539.856 6787758.885, 402539.662 6787758.454, 402539.459 6787758.159, 402539.088 6787757.998, 402538.793 6787758.142, 402538.514 6787758.479, 402538.489 6787758.809, 402538.598 6787759.138, 402538.869 6787759.501, 402538.860 6787759.754, 402538.801 6787760.168, 402538.860 6787760.489, 402539.029 6787760.801, 402539.071 6787761.004, 402539.181 6787761.173, 402539.198..."
2,groundwood,10_1128,1,"POLYGON ((402491.012 6787761.780, 402491.257 6787762.076, 402491.654 6787762.700, 402491.958 6787763.123, 402492.101 6787762.945, 402491.679 6787762.354, 402491.350 6787761.822, 402491.012 6787761.780))"
3,groundwood,10_1128,1,"POLYGON ((402489.802 6787761.863, 402489.760 6787762.124, 402489.642 6787762.749, 402489.447 6787763.644, 402489.110 6787764.649, 402489.110 6787764.995, 402489.591 6787765.029, 402490.013 6787763.577, 402490.047 6787763.230, 402490.283 6787762.361, 402490.300 6787761.863, 402489.802 6787761.863))"
4,groundwood,10_1128,1,"POLYGON ((402507.264 6787760.396, 402507.264 6787760.505, 402507.256 6787760.649, 402507.272 6787760.868, 402507.222 6787761.240, 402507.180 6787761.628, 402507.163 6787762.084, 402507.120 6787762.396, 402507.070 6787762.641, 402506.969 6787762.945, 402506.893 6787763.232, 402506.825 6787763.553, 402506.817 6787763.823, 402506.707 6787764.760, 402506.681 6787765.005, 402506.656 6787765.326, 402506.622 6787765.731, 402506.555 6787766.001, 402506.437 6787766.567, 402506.318 6787767.226, 402506.183 6787768.053, 402506.521 6787768.222, 402506.572 6787767.749, 402506.673 6787767.226, 402506.673..."
...,...,...,...,...
37,groundwood,9_1123,1,"POLYGON ((402731.939 6787926.071, 402731.948 6787926.088, 402731.952 6787926.122, 402731.901 6787926.117, 402731.859 6787926.168, 402731.923 6787926.278, 402731.859 6787926.350, 402731.918 6787926.392, 402731.973 6787926.430, 402732.062 6787926.468, 402732.395 6787926.556, 402732.758 6787926.670, 402732.965 6787926.768, 402733.349 6787926.814, 402733.679 6787926.839, 402733.995 6787926.898, 402734.312 6787926.966, 402734.700 6787927.067, 402734.966 6787927.173, 402735.021 6787927.131, 402735.063 6787927.004, 402734.979 6787926.932, 402734.776 6787926.818, 402734.489 6787926.713, 402734.177..."
38,uprightwood,9_1123,1,"POLYGON ((402685.798 6787933.801, 402685.848 6787933.890, 402686.013 6787934.086, 402686.228 6787934.270, 402686.488 6787934.371, 402686.741 6787934.403, 402686.880 6787934.194, 402686.861 6787933.978, 402686.887 6787933.826, 402687.089 6787933.693, 402687.317 6787933.592, 402687.355 6787933.427, 402687.349 6787933.250, 402687.222 6787932.965, 402687.191 6787933.181, 402687.241 6787933.408, 402687.102 6787933.510, 402686.899 6787933.497, 402686.811 6787933.351, 402686.804 6787933.218, 402686.785 6787933.048, 402686.747 6787932.858, 402686.735 6787932.668, 402686.716 6787932.478, 402686.691..."
39,uprightwood,9_1123,1,"POLYGON ((402658.026 6787937.739, 402658.058 6787937.796, 402658.248 6787938.037, 402658.311 6787938.164, 402658.355 6787938.176, 402658.501 6787937.993, 402658.495 6787937.828, 402658.577 6787937.746, 402658.723 6787937.632, 402658.729 6787937.467, 402658.602 6787937.417, 402658.431 6787937.423, 402658.393 6787937.417, 402658.469 6787937.284, 402658.583 6787937.100, 402658.596 6787937.018, 402658.482 6787936.910, 402658.438 6787937.043, 4

In [164]:
tp_truths = truths.copy()
tp_truths['label'] = tp_truths.layer.apply(lambda row: 1 if row=='uprightwood' else 2)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

100%|██████████| 7017/7017 [01:14<00:00, 94.70it/s] 


In [165]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,3612,2051,5663
uprightwood,505,849,1354
All,4117,2900,7017


In [166]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,1864,2051,3915
uprightwood,571,848,1419
All,2435,2899,5334


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [188]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(2051/5663):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(2051/3915):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.36
Recall for fallen deadwood with IoU threshold of 0.5 is 0.52


In [189]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(849/1354):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(849/1419):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.63
Recall for standing deadwood with IoU threshold of 0.5 is 0.60


In [190]:
print(f'Overall precision with IoU threshold of 0.5 is {(2900/7017):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(2899/5334):.2f}')

Overall precision with IoU threshold of 0.5 is 0.41
Overall recall with IoU threshold of 0.5 is 0.54


## Half patch overlap and edge filtering

In [171]:
spk_res_path = Path('../results/spk_buffered/')
truth_shps = sorted([spk_res_path/'vector_tiles'/f for f in os.listdir(spk_res_path/'vector_tiles')])
spk_buf_raw_shps = sorted([spk_res_path/'raw_preds'/f for f in os.listdir(spk_res_path/'raw_preds')])
rasters = sorted([spk_res_path/'raster_tiles'/f for f in os.listdir(spk_res_path/'raster_tiles')])

In [172]:
for p, t in zip(spk_buf_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.0485**2]
    temp_pred.to_file(spk_res_path/'predicted_vectors'/p.name)

In [173]:
pred_shps = sorted([spk_res_path/'predicted_vectors'/f for f in os.listdir(spk_res_path/'predicted_vectors')])

In [174]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [175]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [176]:
preds.shape, truths.shape

((7157, 4), (5334, 4))

In [177]:
preds.layer.value_counts()

groundwood     5777
uprightwood    1380
Name: layer, dtype: int64

In [178]:
truths.rename(columns={'label':'layer'}, inplace=True)

In [179]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [180]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.458800
uprightwood    0.598393
dtype: float64

In [181]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

spk_coco_eval = GisCOCOeval(spk_res_path, spk_res_path, 
                           None, None, deadwood_categories)
spk_coco_eval.prepare_data(gt_label_col='label')
spk_coco_eval.prepare_eval()
spk_coco_eval.coco_eval.params.maxDets = [1000, 10000]
spk_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=2.17s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.29s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.53s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.591
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.449
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.408
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.223
 Average Recal

In [182]:
tp_truths = truths.copy()
tp_truths['label'] = tp_truths.layer.apply(lambda row: 1 if row=='uprightwood' else 2)
truth_sindex = tp_truths.sindex
fp_preds = preds.copy()
pred_sindex = fp_preds.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds[fp_cols] = fp_preds.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds, pred_sindex),
                                            axis=1, result_type='expand')

100%|██████████| 7157/7157 [02:15<00:00, 52.72it/s] 


In [183]:
pd.crosstab(fp_preds.layer, fp_preds['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,3508,2269,5777
uprightwood,441,939,1380
All,3949,3208,7157


In [184]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,1646,2269,3915
uprightwood,481,938,1419
All,2127,3207,5334


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [193]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(2269/5777):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(2269/3915):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.39
Recall for fallen deadwood with IoU threshold of 0.5 is 0.58


In [191]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(939/1380):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(939/1419):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.68
Recall for standing deadwood with IoU threshold of 0.5 is 0.66


In [194]:
print(f'Overall precision with IoU threshold of 0.5 is {(3208/7157):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(3207/5334):.2f}')

Overall precision with IoU threshold of 0.5 is 0.45
Overall recall with IoU threshold of 0.5 is 0.60


## Overlap, edge filtering and mask merging

In [195]:
merge_outpath = Path('../results/spk_merge/')
if not os.path.exists(merge_outpath):
    shutil.copytree('../results/spk_template/', merge_outpath)

Two iterations of merging is usually enough.

In [ ]:
for r in pred_shps:
    gdf_temp = gpd.read_file(r)
    standing = gdf_temp[gdf_temp.label==1].copy()
    fallen = gdf_temp[gdf_temp.label==2].copy()
    if len(standing) > 0:
        standing = merge_polys(standing, 0.2)
        standing = merge_polys(standing, 0.2)
    if len(fallen) > 0:
        fallen = merge_polys(fallen, 0.2)
        fallen = merge_polys(fallen, 0.2)
    gdf_merged = pd.concat((standing, fallen))
    gdf_merged.to_file(merge_outpath/'predicted_vectors'/r.name, driver='GeoJSON')
    gdf_merged = None
    gdf_temp = None

In [197]:
merged_coco_eval = GisCOCOeval(merge_outpath, merge_outpath, None, None, deadwood_categories)
merged_coco_eval.prepare_data(gt_label_col='label')
merged_coco_eval.prepare_eval()
merged_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=1.73s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.79s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=7.79s).
Accumulating evaluation results...
DONE (t=0.33s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.319
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.571
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.431
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.521
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.391
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.219
 Average Recall     (A

In [198]:
preds.reset_index(drop=True, inplace=True)
standing = merge_polys(preds[preds.label == 1].copy(), 0.2)
fallen = merge_polys(preds[preds.label == 2].copy(), 0.2)
standing = merge_polys(standing, 0.2)
fallen = merge_polys(fallen, 0.2)
preds_merged = pd.concat((standing, fallen))

1380it [00:24, 57.15it/s]
5777it [05:50, 16.49it/s]
1317it [00:16, 79.90it/s]
4633it [03:04, 25.15it/s]


In [199]:
preds_merged['layer'] = preds_merged.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)
preds_merged.layer.value_counts()

groundwood     4473
uprightwood    1312
Name: layer, dtype: int64

In [201]:
tp_truths = truths.copy()
tp_truths['label'] = tp_truths.layer.apply(lambda row: 1 if row=='uprightwood' else 2)
truth_sindex = tp_truths.sindex
fp_preds_merged = preds_merged.copy()
pred_sindex = fp_preds_merged.sindex
tp_truths[tp_cols] = tp_truths.progress_apply(lambda row: is_true_positive(row, fp_preds_merged, pred_sindex), 
                                              axis=1, result_type='expand')
fp_preds_merged[fp_cols] = fp_preds_merged.progress_apply(lambda row: is_false_positive(row, tp_truths, truth_sindex,
                                                                            fp_preds_merged, pred_sindex),
                                            axis=1, result_type='expand')

100%|██████████| 5785/5785 [01:38<00:00, 58.54it/s] 


In [202]:
pd.crosstab(fp_preds_merged.layer, fp_preds_merged['FP_0.5'], margins=True)

FP_0.5,FP,TP,All
layer,,,
groundwood,2310,2163,4473
uprightwood,408,904,1312
All,2718,3067,5785


In [203]:
pd.crosstab(tp_truths.layer, tp_truths['TP_0.5'], margins=True)

TP_0.5,FN,TP,All
layer,,,
groundwood,1752,2163,3915
uprightwood,516,903,1419
All,2268,3066,5334


$Precision = \frac{tp}{tp+fp}, Recall = \frac{tp}{tp+fn}$

In [148]:
print(f'Precision for fallen deadwood with IoU threshold of 0.5 is {(2163/4473):.2f}')
print(f'Recall for fallen deadwood with IoU threshold of 0.5 is {(2163/3915):.2f}')

Precision for fallen deadwood with IoU threshold of 0.5 is 0.55
Recall for fallen deadwood with IoU threshold of 0.5 is 0.72


In [204]:
print(f'Precision for standing deadwood with IoU threshold of 0.5 is {(904/1312):.2f}')
print(f'Recall for standing deadwood with IoU threshold of 0.5 is {(904/1419):.2f}')

Precision for standing deadwood with IoU threshold of 0.5 is 0.69
Recall for standing deadwood with IoU threshold of 0.5 is 0.64


In [205]:
print(f'Overall precision with IoU threshold of 0.5 is {(3067/5785):.2f}')
print(f'Overall recall with IoU threshold of 0.5 is {(3067/5334):.2f}')

Overall precision with IoU threshold of 0.5 is 0.53
Overall recall with IoU threshold of 0.5 is 0.57


In [162]:
preds_merged.to_file('../results/sudenpesankangas/spk_merged.geojson')